In [ ]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import os

#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
def generate_matrix(n, dirichlet_param=0.1):
    alpha = [dirichlet_param for _ in range(n)]
    return np.array([np.random.dirichlet(alpha) for _ in range(n)])

In [ ]:
nl_dict = {3: [1, 2, 3, 5], 5: [1, 3, 5, 10], 10: [1, 5, 10, 15]}

# Input params
mean = 0.
std = 1.
lr = 0.01
iter_ = 20 # Number of repeatitions per GT matrix and per n,l-parameter
n_gt = 10  # Number of GT matrices to generate

n_iter = 30000 # Optimization iterations
min_iter = 2000 # Minimum number of optimization iteration
convergence_tol = 0. # Convergence threshold

In [ ]:
# Graph
def build_graph(n, l):
    
    A_gt = tf.placeholder(dtype=tf.float32, shape=[n, n])
    init_normal = tf.random_normal_initializer(mean, stddev=std)
    
    # Approach (i) - normAbsLin factorization with normalization
    with tf.variable_scope("%d%d_lin" % (n, l), reuse=tf.AUTO_REUSE):
        V_lin = tf.get_variable("V", shape=[n, l], dtype=tf.float32, initializer=init_normal)
        W_lin = tf.get_variable("W", shape=[l, n], dtype=tf.float32, initializer=init_normal)
    
    A_tilde_lin = tf.math.abs(tf.matmul(V_lin, W_lin))
    A_tilde_lin /= tf.reduce_sum(A_tilde_lin, axis=1)[:, None]
    A_sum_lin = tf.reduce_sum(A_tilde_lin, axis=1)
    loss_lin = tf.norm(A_gt - A_tilde_lin)
    loss_norm_lin = tf.norm(A_gt - A_tilde_lin) / tf.norm(A_gt)
    #opt_lin = tf.train.GradientDescentOptimizer(lr).minimize(loss_norm_lin)
    
    with tf.variable_scope("%d%d_lin" % (n, l), reuse=tf.AUTO_REUSE):
        opt_lin = tf.train.AdamOptimizer(lr).minimize(loss_norm_lin)
    
    # Approach (ii) - Softmax factorization
    with tf.variable_scope("%d%d_sm" % (n, l), reuse=tf.AUTO_REUSE):
        V_sm = tf.Variable(V_lin.initialized_value(), dtype=tf.float32, name="V")#tf.get_variable("V", shape=[n, l], dtype=tf.float32, initializer=init_normal)
        W_sm = tf.Variable(W_lin.initialized_value(), dtype=tf.float32, name="W")

    A_tilde_sm = tf.math.softmax(tf.matmul(V_sm, W_sm))
    A_sum_sm = tf.reduce_sum(A_tilde_sm, axis=1)
    loss_sm = tf.norm(A_gt - A_tilde_sm)
    loss_norm_sm = tf.norm(A_gt - A_tilde_sm) / tf.norm(A_gt)
    #opt_sm = tf.train.GradientDescentOptimizer(lr).minimize(loss_norm_sm)
    
    with tf.variable_scope("%d%d_sm" % (n, l), reuse=tf.AUTO_REUSE):
        opt_sm = tf.train.AdamOptimizer(lr).minimize(loss_norm_sm)
    
    #opt = tf.train.GradientDescentOptimizer(lr).minimize(loss)
    init = tf.global_variables_initializer()
    
    return init, A_gt, A_tilde_lin, A_tilde_sm, opt_lin, opt_sm, loss_norm_lin, loss_norm_sm


In [ ]:
exp_dir = 'matrix_fit_exp_adam' + datetime.now().strftime('%Y%m%d_%H-%M-%S')
os.mkdir('./%s' % exp_dir)
os.mkdir('./%s/savepoints' % exp_dir) 

gt_mats = dict()
losses_sm, losses_relu, losses_lin = dict(), dict(), dict()
mean_losses_sm, mean_losses_relu, mean_losses_lin = dict(), dict(), dict()
std_losses_sm, std_losses_relu, std_losses_lin = dict(), dict(), dict()
optimized_mat_sm, optimized_mat_relu, optimized_mat_lin = dict(), dict(), dict()
init_loss_sm, init_loss_relu, init_loss_lin = dict(), dict(), dict()

sess = tf.Session()

for i in range(n_gt):  # Do the training for n_gt matrices

    for n, l_list in nl_dict.iteritems():
        
        _A_inp = generate_matrix(n)
        gt_mats[(i, n)] = _A_inp

        for l in l_list:

            losses_sm[(i, n, l)], losses_lin[(i, n, l)] = [], []
            optimized_mat_sm[(i, n, l)], optimized_mat_lin[(i, n, l)] = [], []
            init_loss_sm[(i, n, l)], init_loss_lin[(i, n, l)] = [], []
            
            init, A, A_tilde_lin, A_tilde_sm, opt_lin, opt_sm, loss_norm_lin, loss_norm_sm = build_graph(n, l)
            feed_dict_ = {A: _A_inp}
            print("build graph", i, n, l)
            
            prev_loss_lin, prev_loss_sm = 0., 0.

            for _ in range(iter_):

                sess.run(init)  # Re-initialize variables (re-draws U, V)
                prev_loss_lin = sess.run([loss_norm_lin], feed_dict=feed_dict_)
                prev_loss_sm = sess.run([loss_norm_sm], feed_dict_)
                
                init_loss_lin[(i, n, l)].append(prev_loss_lin)
                init_loss_sm[(i, n, l)].append(prev_loss_sm)

                for step in range(n_iter):
                    
                    if step < min_iter:
                        _, _ = sess.run([opt_lin, opt_sm], feed_dict=feed_dict_)
                    else:
                        opt = []
                        if not ((cur_loss_lin <= prev_loss_lin) and (prev_loss_lin - cur_loss_lin <= convergence_tol)):
                            opt.append(opt_lin)
                        if not ((cur_loss_sm <= prev_loss_sm) and (prev_loss_sm - cur_loss_sm <= convergence_tol)):
                            opt.append(opt_sm)
                        
                        if len(opt) < 1:  # All converged
                            print("All converged")
                            break
                        else:
                            sess.run(opt, feed_dict=feed_dict_)
                        
                    if step >= 2:
                        prev_loss_lin, prev_loss_sm = cur_loss_lin, cur_loss_sm
                    cur_loss_lin, cur_loss_sm = sess.run([loss_norm_lin, loss_norm_sm], feed_dict=feed_dict_)

                    if step % 5000 == 0:
                        print(cur_loss_lin, cur_loss_sm)

                optimized_mat_lin[(i, n, l)].append(sess.run([A_tilde_lin]))
                optimized_mat_sm[(i, n, l)].append(sess.run([A_tilde_sm]))

                print(i, n, l, cur_loss_lin, cur_loss_sm)
                losses_lin[(i, n, l)].append(cur_loss_lin)
                losses_sm[(i, n, l)].append(cur_loss_sm)

            mean_losses_sm[(i, n, l)], mean_losses_lin[(i, n, l)] = np.mean(losses_sm[(i, n, l)]), np.mean(losses_lin[(i, n, l)])
            std_losses_sm[(i, n, l)], std_losses_lin[(i, n, l)] = np.std(losses_sm[(i, n, l)]), np.std(losses_lin[(i, n, l)])
        
    np.save('%s/savepoints/losses_lin.npy' % exp_dir, losses_lin)
    np.save('%s/savepoints/losses_sm.npy'% exp_dir, losses_sm)
    np.save('%s/savepoints/mean_losses_lin.npy'% exp_dir, mean_losses_lin)
    np.save('%s/savepoints/mean_losses_sm.npy'% exp_dir, mean_losses_sm)
    np.save('%s/savepoints/std_losses_sm.npy'% exp_dir, std_losses_sm)
    np.save('%s/savepoints/std_losses_lin.npy'% exp_dir, std_losses_lin)
    np.save('%s/savepoints/optimized_mat_lin.npy'% exp_dir, optimized_mat_lin)
    np.save('%s/savepoints/optimized_mat_sm.npy'% exp_dir, optimized_mat_sm)
    np.save('%s/savepoints/init_loss_sm.npy'% exp_dir, init_loss_sm)
    np.save('%s/savepoints/init_loss_lin.npy'% exp_dir, init_loss_lin)
    np.save('%s/savepoints/gt_mats.npy'% exp_dir,gt_mats)
        
        
np.save('%s/losses_lin.npy' % exp_dir, losses_lin)
np.save('%s/losses_sm.npy'% exp_dir, losses_sm)
                   
np.save('%s/mean_losses_lin.npy'% exp_dir, mean_losses_lin)
np.save('%s/mean_losses_sm.npy'% exp_dir, mean_losses_sm)
                   
np.save('%s/std_losses_sm.npy'% exp_dir, std_losses_sm)
np.save('%s/std_losses_lin.npy'% exp_dir, std_losses_lin)
                   
np.save('%s/optimized_mat_lin.npy'% exp_dir, optimized_mat_lin)
np.save('%s/optimized_mat_sm.npy'% exp_dir, optimized_mat_sm)
                   
np.save('%s/init_loss_sm.npy'% exp_dir, init_loss_sm)
np.save('%s/init_loss_lin.npy'% exp_dir, init_loss_lin)

np.save('%s/gt_mats.npy'% exp_dir,gt_mats)
                   
print("Done")
print(exp_dir)